In [5]:
%pip install faker


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col, expr
from pyspark.sql.types import StringType, DateType
from faker import Faker
import pandas as pd

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("GenerateFakeData") \
    .config("spark.sql.shuffle.partitions", "500") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "16") \
    .getOrCreate()

ModuleNotFoundError: No module named 'pandas'

In [7]:

fake = Faker()

# UDFs for generating fake data
def generate_first_name():
    return fake.first_name()

def generate_last_name():
    return fake.last_name()

def generate_address():
    return fake.address()

def generate_dob():
    return fake.date_of_birth(minimum_age=18, maximum_age=90).strftime("%Y-%m-%d")

first_name_udf = udf(generate_first_name, StringType())
last_name_udf = udf(generate_last_name, StringType())
address_udf = udf(generate_address, StringType())
dob_udf = udf(generate_dob, StringType())

NameError: name 'Faker' is not defined

In [8]:
first_name_udf = udf(generate_first_name, StringType())
last_name_udf = udf(generate_last_name, StringType())
address_udf = udf(generate_address, StringType())
dob_udf = udf(generate_dob, StringType())

# Create a DataFrame with the required number of rows
num_records = 1000  # Adjust based on the size needed
df = spark.range(num_records) \
    .select(
        first_name_udf().alias("first_name"),
        last_name_udf().alias("last_name"),
        address_udf().alias("address"),
        dob_udf().alias("dob")
    )



NameError: name 'generate_first_name' is not defined

In [9]:
df = df.repartition(100)  # Increase partitions if necessary

# Define output path inside the container
output_path = "/opt/notebooks/output/output2.csv"  # Use a path inside the container

# Write the DataFrame to CSV
df.write.csv(output_path, header=True, mode="overwrite")

print(f"Data saved at: {output_path}")

NameError: name 'df' is not defined

In [10]:
# Path to the CSV file in DBFS


# Read the CSV file from DBFS
df = spark.read.csv(output_path, header=True, inferSchema=True)

# Define the masking functions
def mask_name(name):
    return name[0] + "*" * (len(name) - 1)  # Mask all but the first character

def mask_address(address):
    return "Address Masked"

# Register UDFs for masking
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

mask_name_udf = udf(mask_name, StringType())
mask_address_udf = udf(mask_address, StringType())

# Apply masking to the DataFrame
masked_df = df.withColumn("first_name", mask_name_udf(col("first_name"))) \
              .withColumn("last_name", mask_name_udf(col("last_name"))) \
              .withColumn("address", mask_address_udf(col("address")))
masked_df.show(50)


NameError: name 'spark' is not defined

In [ ]:
masked_df.coalesce(1)

# Define output path inside the container
output_path = "/opt/notebooks/output/masked_output.csv"  # Use a path inside the container

# Write the DataFrame to CSV
df.write.csv(output_path, header=True, mode="overwrite")

print(f"Data saved at: {output_path}")